# 多线程

不必让所有代码等待直到`time.sleep`函数完成，可以使用Python的threading模块，在单独的线程中执行延迟或者安排的代码。这个单独的线程将因为time.sleep()调用而暂停。同时，程序可以在原来的线程中做其他工作。

这意味着我们要在新线程中调用的函数是`wantSleep()`。请注意，关键字参数是`target=wantSleep`，
而不是`target=takeANap()`。这是因为你想将`wantSleep()`函数本身作为参数，而不是调用`wantSleep()`，并传入它的返回值。

In [3]:
import threading
import time

print('start of program')

def wantSleep():
    time.sleep(3)
    print('Wake up')
    
# 生成Thread对象得到单独的线程
threadObj = threading.Thread(target=wantSleep) 
threadObj.start()

print('end of program')

start of program
end of program
Wake up


## 向进程的目标函数传递参数

该print()调用有3 个常规参数：'Cats'、 'Dogs'和'Frogs'，以及一个关键字参数：sep=' & '。常规参数可以作为一个列表，传递给threading.Thread()中的args 关键字参数。关键字参数可以作为一个字典，传递给threading.Thread()中的kwargs 关键字参数。

In [8]:
threadObj = threading.Thread(target=print, args=['Cats','Dogs','Frogs'], kwargs={'sep':' & '})
threadObj.start()

Cats & Dogs & Frogs


下面创建新线程调用print()的方法是不正确的：
```python
threadObj = threading.Thread(target=print('Cats', 'Dogs', 'Frogs', sep=' & '))
```
这行代码最终会调用print()函数，将它的返回值（print()的返回值总是无）作为
target 关键字参数。它没有传递print()函数本身。如果要向新线程中的函数传递参数，
就使用threading.Thread()函数的args 和kwargs 关键字参数。

# 并发

主线程正常执行，同时我们创建的其他线程下载漫画。但是假定主线程中有一
些代码，你希望所有下载线程完成后再执行。调用Thread 对象join()方法将阻塞，
直到该线程完成。利用一个for 循环，遍历downloadThreads 列表中的所有Thread
对象，主线程可以调用其他每个线程的join()方法。

In [13]:
import requests, os, bs4, threading
os.makedirs('xkcd', exist_ok=True) # store comics in ./xkcd

def downloadXkcd(startComic, endComic):
    for urlNumber in range(startComic, endComic):
        # Download the page.
        print('Downloading page http://xkcd.com/%s...' % (urlNumber))
        res = requests.get('http://xkcd.com/%s' % (urlNumber))
        res.raise_for_status()
        
        # generate an instance according to HTML
        soup = bs4.BeautifulSoup(res.text) 
        
        # Find the URL of the comic image.
        comicElem = soup.select('#comic img')
        
        if comicElem == []:
            print('Could not find comic image.')
        else:
            comicUrl = comicElem[0].get('src')
        # Download the image.
        print('Downloading image %s...' % (comicUrl))
        res = requests.get(comicUrl)
        res.raise_for_status()
        
        # Save the image to ./xkcd.
        imageFile = open(os.path.join('xkcd', os.path.basename(comicUrl)), 'wb')
        for chunk in res.iter_content(100000):
            imageFile.write(chunk)
        imageFile.close()
        

# Create and start the Thread objects.
downloadThreads = [] # a list of all the Thread objects
for i in range(0, 1400, 100): # loops 14 times, creates 14 threads
    downloadThread = threading.Thread(target=downloadXkcd, args=(i, i + 99))
    downloadThreads.append(downloadThread)
    downloadThread.start()
    
# Wait for all threads to end.
for downloadThread in downloadThreads:
    downloadThread.join()
print('Done.')

/home/zjw/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 884 of the file /home/zjw/anaconda3/lib/python3.6/threading.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
Exception in thread Thread-23:
Traceback (most recent call last):
  File "/home/zjw/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/zjw/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-13-21ec47034ebd>", line 23, in downloadXkcd
    res = re

Exception in thread Thread-21:
Traceback (most recent call last):
  File "/home/zjw/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/zjw/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-13-21ec47034ebd>", line 23, in downloadXkcd
    res = requests.get(comicUrl)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/api.py", line 72, in get
    return request('get', url, params=params, **kwargs)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/api.py", line 58, in request
    return session.request(method=method, url=url, **kwargs)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/sessions.py", line 494, in request
    prep = self.prepare_request(req)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/sessions.py", line 437, in prepare_request
    hooks=merge_hooks(request.hooks, self.hooks),
  File "/home/zj

Exception in thread Thread-24:
Traceback (most recent call last):
  File "/home/zjw/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/zjw/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-13-21ec47034ebd>", line 23, in downloadXkcd
    res = requests.get(comicUrl)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/api.py", line 72, in get
    return request('get', url, params=params, **kwargs)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/api.py", line 58, in request
    return session.request(method=method, url=url, **kwargs)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/sessions.py", line 494, in request
    prep = self.prepare_request(req)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/sessions.py", line 437, in prepare_request
    hooks=merge_hooks(request.hooks, self.hooks),
  File "/home/zj

Exception in thread Thread-22:
Traceback (most recent call last):
  File "/home/zjw/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/zjw/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-13-21ec47034ebd>", line 23, in downloadXkcd
    res = requests.get(comicUrl)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/api.py", line 72, in get
    return request('get', url, params=params, **kwargs)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/api.py", line 58, in request
    return session.request(method=method, url=url, **kwargs)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/sessions.py", line 494, in request
    prep = self.prepare_request(req)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/sessions.py", line 437, in prepare_request
    hooks=merge_hooks(request.hooks, self.hooks),
  File "/home/zj

Exception in thread Thread-15:
Traceback (most recent call last):
  File "/home/zjw/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/zjw/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-13-21ec47034ebd>", line 23, in downloadXkcd
    res = requests.get(comicUrl)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/api.py", line 72, in get
    return request('get', url, params=params, **kwargs)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/api.py", line 58, in request
    return session.request(method=method, url=url, **kwargs)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/sessions.py", line 494, in request
    prep = self.prepare_request(req)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/sessions.py", line 437, in prepare_request
    hooks=merge_hooks(request.hooks, self.hooks),
  File "/home/zj

Exception in thread Thread-18:
Traceback (most recent call last):
  File "/home/zjw/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/zjw/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-13-21ec47034ebd>", line 23, in downloadXkcd
    res = requests.get(comicUrl)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/api.py", line 72, in get
    return request('get', url, params=params, **kwargs)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/api.py", line 58, in request
    return session.request(method=method, url=url, **kwargs)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/sessions.py", line 494, in request
    prep = self.prepare_request(req)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/sessions.py", line 437, in prepare_request
    hooks=merge_hooks(request.hooks, self.hooks),
  File "/home/zj

Exception in thread Thread-16:
Traceback (most recent call last):
  File "/home/zjw/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/zjw/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-13-21ec47034ebd>", line 23, in downloadXkcd
    res = requests.get(comicUrl)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/api.py", line 72, in get
    return request('get', url, params=params, **kwargs)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/api.py", line 58, in request
    return session.request(method=method, url=url, **kwargs)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/sessions.py", line 494, in request
    prep = self.prepare_request(req)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/sessions.py", line 437, in prepare_request
    hooks=merge_hooks(request.hooks, self.hooks),
  File "/home/zj

Exception in thread Thread-26:
Traceback (most recent call last):
  File "/home/zjw/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/zjw/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-13-21ec47034ebd>", line 23, in downloadXkcd
    res = requests.get(comicUrl)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/api.py", line 72, in get
    return request('get', url, params=params, **kwargs)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/api.py", line 58, in request
    return session.request(method=method, url=url, **kwargs)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/sessions.py", line 494, in request
    prep = self.prepare_request(req)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/sessions.py", line 437, in prepare_request
    hooks=merge_hooks(request.hooks, self.hooks),
  File "/home/zj

Done.


Exception in thread Thread-19:
Traceback (most recent call last):
  File "/home/zjw/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/zjw/anaconda3/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-13-21ec47034ebd>", line 23, in downloadXkcd
    res = requests.get(comicUrl)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/api.py", line 72, in get
    return request('get', url, params=params, **kwargs)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/api.py", line 58, in request
    return session.request(method=method, url=url, **kwargs)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/sessions.py", line 494, in request
    prep = self.prepare_request(req)
  File "/home/zjw/anaconda3/lib/python3.6/site-packages/requests/sessions.py", line 437, in prepare_request
    hooks=merge_hooks(request.hooks, self.hooks),
  File "/home/zj

4